In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import src.geo_functions as gf

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("geospatial")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'geospatial')

In [3]:
lisbon = db.get_collection("lisbon")
london = db.get_collection("london")
madrid = db.get_collection("madrid")

In [ ]:
# Creamos el  index 2dsphere desde pymongo
#db.lisbon.create_index([("location", GEOSPHERE)])
#db.london.create_index([("location", GEOSPHERE)])
#db.madrid.create_index([("location", GEOSPHERE)])

GEOQUERIES : NEAR

# LONDON 

In [4]:
london_ = gf.geocode("St Paul´s Cathedral, London")

In [5]:
london_

{'type': 'Point', 'coordinates': ['51.51421', '-0.05688']}

In [6]:
london_coord = gf.type_point([51.51421, -0.05688])

In [7]:
london_coord

{'type': 'Point', 'coordinates': [51.51421, -0.05688]}

In [12]:
# Km que quiero  caminar
metros = 1000

In [13]:
query = {"location": {"$near": {"$geometry": london_coord, "$maxDistance": metros}}}

In [16]:
london_query = list(london.find(query))

In [17]:
london_query

[{'_id': ObjectId('61902cdda80a1fd067e41923'),
  'nombre': 'Shadwell London Overground Station',
  'latitud': 51.51145248356203,
  'longitud': -0.05667416171207461,
  'location': {'type': 'Point',
   'coordinates': [51.51145248356203, -0.05667416171207461]}},
 {'_id': ObjectId('61902cb1a80a1fd067e418b0'),
  'nombre': 'LHT Urban Bar',
  'latitud': 51.51925520009031,
  'longitud': -0.0589987201191938,
  'location': {'type': 'Point',
   'coordinates': [51.51925520009031, -0.0589987201191938]}},
 {'_id': ObjectId('61902cdda80a1fd067e4191a'),
  'nombre': 'Whitechapel Railway Station (ZLW)',
  'latitud': 51.5195054401361,
  'longitud': -0.0612187385559082,
  'location': {'type': 'Point',
   'coordinates': [51.5195054401361, -0.0612187385559082]}},
 {'_id': ObjectId('61902cdda80a1fd067e4191c'),
  'nombre': 'Whitechapel London Underground and London Overground Station',
  'latitud': 51.519477137040596,
  'longitud': -0.06151677976888857,
  'location': {'type': 'Point',
   'coordinates': [51.51